# PDI - Rebalancing data for July 2022

## Setting

In [23]:
import pandas as pd
import numpy as np
import requests

In [24]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [25]:
from defillama import DefiLlama
llama = DefiLlama()

In [26]:
import plotly.express as px
import plotly.graph_objects as go

In [27]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics** 
    1. The project must have a listing on Defi Llama
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.

1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have at least $5mm of onchain liquidity on a single pair.
    1. The token must have shown consistent DeFi liquidity on Ethereum.

1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

#### DefiLlama

In [28]:
protocols = llama.get_all_protocols()

In [29]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['symbol'] != '-':
        symbol = ''.join(c for c in pr['symbol'] if c.isalnum()).upper()
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': pr['address'].lower()
        }

In [30]:
top_defi_tokens['XSUSHI'] = {
    'symbol': 'XSUSHI',
    'address': "0x8798249c2e607446efb7ad49ec89dd1865ff4272",
}

#### CoinGecko

##### Tokens addresses

In [31]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [32]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = ''.join(c for c in coin['symbol'] if c.isalnum()).upper()
        address = coin['platforms']['ethereum'].lower()
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token

In [33]:
tokens['USDC'], tokens['WETH']

({'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'})

In [34]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        print(token)
        exclude_symbols.append(token['symbol'])

{'symbol': 'MULTI', 'address': '0x9fb9a33956351cf4fa040f65a13b835a3c8764e3'}
{'symbol': 'IB', 'address': 'fantom:0x00a35fd824c717879bf370e70ac6868b95870dfb'}
{'symbol': 'COW', 'address': 'heco:0x80861a817106665bca173db6ac2ab628a738c737'}
{'symbol': 'ABR', 'address': 'celo:0x6e512bfc33be36f2666754e996ff103ad1680cc9'}
{'symbol': 'FLAG', 'address': '0x57dd84042ec9507963016596a34edad42f7e4cce '}
{'symbol': 'RSR', 'address': '0x8762db106b2c2a0bccb3a80d1ed41273552616e8'}
{'symbol': 'FHM', 'address': '0xfa1fbb8ef55a4855e5688c0ee13ac3f202486286'}
{'symbol': 'SOLV', 'address': '0x256f2d67e52fe834726d2ddcd8413654f5eb8b53'}
{'symbol': 'SGT', 'address': '0x84810bcf08744d5862b8181f12d17bfd57d3b078'}
{'symbol': 'GEAR', 'address': '0xba3335588d9403515223f109edc4eb7269a9ab5d'}
{'symbol': 'SWPR', 'address': 'arbitrum:0xde903e2712288a1da82942dddf2c20529565ac30'}
{'symbol': 'MFO', 'address': 'bsc:0xb46049c79d77ff1d555a67835fba6978536581af'}
{'symbol': 'USDS', 'address': 'arbitrum:0xd74f5255d557944cf7dd0e

In [35]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

In [36]:
# KEEP has this address in defillama, but etherscan says this address belongs to tBTC now
for token in top_defi_tokens.values():
    if token['address'] == "0x8daebade922df735c38c80c7ebd708af50815faa":
        print(token)
del top_defi_tokens['KEEP']

{'symbol': 'KEEP', 'address': '0x8daebade922df735c38c80c7ebd708af50815faa'}


##### Market data

In [37]:
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc')
for i in range(2, 20):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

In [38]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())

In [39]:
tokens_market_data.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
BTC,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,20089.000000,383307698147,1,4.218709e+11,2.096097e+10,20826.000,...,2.100000e+07,21000000.0,69045.00,-70.90426,2021-11-10T14:24:11.849Z,67.810000,29525.99097,2013-07-06T00:00:00.000Z,None,2022-06-29T14:39:58.942Z
ETH,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1112.550000,134980202599,2,NaN,1.438748e+10,1203.710,...,NaN,NaN,4878.26,-77.19719,2021-11-10T14:24:19.604Z,0.432979,256813.37845,2015-10-20T00:00:00.000Z,"{'times': 73.02862688682394, 'currency': 'btc'...",2022-06-29T14:40:59.221Z
USDT,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999143,66767655514,3,NaN,5.360162e+10,1.004,...,6.682492e+10,NaN,1.32,-24.48442,2018-07-24T00:00:00.000Z,0.572521,74.51641,2015-03-02T00:00:00.000Z,None,2022-06-29T14:36:22.928Z
USDC,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.001000,55645129055,4,NaN,4.989107e+09,1.004,...,1.286780e+03,NaN,1.17,-14.66611,2019-05-08T00:40:28.300Z,0.891848,12.20700,2021-05-19T13:14:05.611Z,None,2022-06-29T14:40:12.301Z
BNB,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,218.250000,35702173903,5,3.610446e+10,1.282112e+09,238.060,...,1.632770e+08,165116760.0,686.31,-68.13958,2021-05-10T07:24:17.097Z,0.039818,549053.27364,2017-10-19T00:00:00.000Z,None,2022-06-29T14:41:17.817Z


### Liquidity check

In [40]:
dex_subgraph_urls = {
    'univ2': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
    'sushi': "https://api.thegraph.com/subgraphs/name/zippoxer/sushiswap-subgraph-fork",
} 

In [41]:
base_tokens = [tokens['USDC'], tokens['WETH']]
base_tokens

[{'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}]

In [42]:
query = """
    query pairs ($token0_ids: [ID!]!, $token1_ids: [ID!]!) {
        pairs (first:1000, where: {token0_in: $token0_ids, token1_in: $token1_ids}){
            id
            token0 {
              id
            }
            token1 {
              id
            }
            reserveUSD
        }
    }
    """

In [43]:
base_tokens_ids = [token['address'] for token in base_tokens]
tokens_ids = [token['address'] for symb, token in top_defi_tokens.items()]

In [44]:
rows = {token['symbol']: {} for token in top_defi_tokens.values()}
for dex_name, url in dex_subgraph_urls.items():
    request = requests.post(url, json={'query': query, 'variables': {'token0_ids': base_tokens_ids, 'token1_ids': tokens_ids}})
    pairs1 = request.json()['data']['pairs']

    request = requests.post(url, json={'query': query, 'variables': {'token0_ids': tokens_ids, 'token1_ids': base_tokens_ids}})
    pairs2 = request.json()['data']['pairs']
    
    pairs = pairs1 + pairs2
    
    print(f"{dex_name}: {len(pairs)} pairs")
    
    for pair in pairs:
        if pair['token0']['id'] in base_tokens_ids:
            base_token = tokens_by_addr[pair['token0']['id']]
            token = tokens_by_addr[pair['token1']['id']]
        else:
            base_token = tokens_by_addr[pair['token1']['id']]
            token = tokens_by_addr[pair['token0']['id']]
        
        colname = f"{dex_name}_{base_token['symbol']}" 
        
        rows[token['symbol']][colname] = float(pair['reserveUSD'])

univ2: 270 pairs
sushi: 148 pairs


In [45]:
liquidities = pd.DataFrame.from_dict(rows, orient='index')
liq_check = (liquidities > 5e6).any(axis=1).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [46]:
lq_passed = tokens_market_data[
    tokens_market_data.index.isin(liq_check_passed.index)
]

In [47]:
lq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
WBTC,wrapped-bitcoin,wbtc,Wrapped Bitcoin,https://assets.coingecko.com/coins/images/7598...,20079.000000,5303143151,16,5.303143e+09,301530275.0,20805.000000,...,2.639788e+05,2.639788e+05,70643.00,-71.56528,2021-11-10T14:40:19.650Z,3139.170000,539.88987,2019-04-02T00:00:00.000Z,None,2022-06-29T14:41:22.795Z
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.090000,2323227432,27,5.089333e+09,132514384.0,5.330000,...,1.000000e+09,1.000000e+09,44.92,-88.73185,2021-05-03T05:25:04.822Z,1.030000,391.31993,2020-09-17T01:20:38.214Z,None,2022-06-29T14:41:11.895Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,60.820000,845436529,55,9.730959e+08,143271249.0,67.000000,...,1.600000e+07,1.600000e+07,661.69,-90.79902,2021-05-18T21:19:59.514Z,26.020000,133.95307,2020-11-05T09:20:11.928Z,None,2022-06-29T14:40:58.713Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.078000,207770559,154,2.694270e+08,53670652.0,1.150000,...,2.429169e+08,2.500000e+08,23.38,-95.38790,2021-03-13T23:44:36.774Z,0.475381,126.79603,2020-11-04T14:53:53.560Z,None,2022-06-29T14:40:51.479Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,5686.810000,179708728,170,2.087212e+08,51114765.0,6030.910000,...,3.666600e+04,3.666600e+04,90787.00,-93.71029,2021-05-12T00:29:37.713Z,31.650000,17944.58693,2020-07-18T12:26:27.150Z,None,2022-06-29T14:41:41.840Z
SYN,synapse-2,syn,Synapse,https://assets.coingecko.com/coins/images/1802...,0.712794,132178992,202,1.781994e+08,3512128.0,0.836028,...,1.849078e+08,2.500000e+08,4.92,-85.50157,2021-10-24T00:24:41.805Z,0.389600,82.92047,2021-09-21T21:19:58.123Z,None,2022-06-29T14:41:37.591Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,23.350000,31522511,518,5.591154e+07,3278032.0,25.080000,...,2.074262e+06,2.393060e+06,2066.20,-98.87053,2021-03-20T19:33:19.740Z,17.340000,34.60576,2022-06-18T20:12:02.334Z,None,2022-06-29T14:41:40.822Z
BOND,barnbridge,bond,BarnBridge,https://assets.coingecko.com/coins/images/1281...,2.820000,19631749,632,2.840075e+07,32196158.0,3.320000,...,1.000000e+07,1.000000e+07,185.69,-98.45535,2020-10-27T00:36:32.813Z,2.200000,30.51782,2022-06-18T20:12:03.498Z,None,2022-06-29T14:41:22.406Z
TOKE,tokemak,toke,Tokemak,https://assets.coingecko.com/coins/images/1749...,1.160000,18300668,658,1.155901e+08,377698.0,1.270000,...,1.000000e+08,1.000000e+08,79.02,-98.53323,2021-11-04T00:35:29.799Z,1.150000,0.66507,2022-06-29T13:48:07.423Z,None,2022-06-29T14:40:54.024Z


### Non quantative checks

In [48]:
exclude_list = [
    {
        'symbol': 'WBTC',
        'reason': 'Criteria 1.C'
    },
    {
        'symbol': 'XSUSHI',
        'reason': 'Duplicates SUSHI'
    },
    {
        'symbol': 'ERN',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'NFTX',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'FXS',
        'reason': 'Criteria 1.G'
    },
    {
        'symbol': 'RGT',
        'reason': 'Deprecated asset'
    },
    {
        'symbol': 'SUPER',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'BOND',
        'reason': 'Criteria 1.I'
    }
]
exclude_symbols = [token['symbol'] for token in exclude_list]

In [49]:
nq_passed = lq_passed[
    ~lq_passed.index.isin(exclude_symbols)
]

In [50]:
nq_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.090000,2323227432,27,5.089333e+09,132514384.0,5.330000,...,1.000000e+09,1.000000e+09,44.92,-88.73185,2021-05-03T05:25:04.822Z,1.030000,391.31993,2020-09-17T01:20:38.214Z,None,2022-06-29T14:41:11.895Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,60.820000,845436529,55,9.730959e+08,143271249.0,67.000000,...,1.600000e+07,1.600000e+07,661.69,-90.79902,2021-05-18T21:19:59.514Z,26.020000,133.95307,2020-11-05T09:20:11.928Z,None,2022-06-29T14:40:58.713Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.078000,207770559,154,2.694270e+08,53670652.0,1.150000,...,2.429169e+08,2.500000e+08,23.38,-95.38790,2021-03-13T23:44:36.774Z,0.475381,126.79603,2020-11-04T14:53:53.560Z,None,2022-06-29T14:40:51.479Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,5686.810000,179708728,170,2.087212e+08,51114765.0,6030.910000,...,3.666600e+04,3.666600e+04,90787.00,-93.71029,2021-05-12T00:29:37.713Z,31.650000,17944.58693,2020-07-18T12:26:27.150Z,None,2022-06-29T14:41:41.840Z
SYN,synapse-2,syn,Synapse,https://assets.coingecko.com/coins/images/1802...,0.712794,132178992,202,1.781994e+08,3512128.0,0.836028,...,1.849078e+08,2.500000e+08,4.92,-85.50157,2021-10-24T00:24:41.805Z,0.389600,82.92047,2021-09-21T21:19:58.123Z,None,2022-06-29T14:41:37.591Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,23.350000,31522511,518,5.591154e+07,3278032.0,25.080000,...,2.074262e+06,2.393060e+06,2066.20,-98.87053,2021-03-20T19:33:19.740Z,17.340000,34.60576,2022-06-18T20:12:02.334Z,None,2022-06-29T14:41:40.822Z
TOKE,tokemak,toke,Tokemak,https://assets.coingecko.com/coins/images/1749...,1.160000,18300668,658,1.155901e+08,377698.0,1.270000,...,1.000000e+08,1.000000e+08,79.02,-98.53323,2021-11-04T00:35:29.799Z,1.150000,0.66507,2022-06-29T13:48:07.423Z,None,2022-06-29T14:40:54.024Z


### Retrieve historical data & livetime check

In [51]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in nq_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 365:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < 365 days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 365:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < 365 days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Excluding SYN, prices data available only for 304 < 365 days
Excluding TOKE, prices data available only for 327 < 365 days


In [52]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

In [53]:
lt_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.090,2323227432,27,5.089333e+09,132514384.0,5.33,...,1.000000e+09,1.000000e+09,44.92,-88.73185,2021-05-03T05:25:04.822Z,1.030000,391.31993,2020-09-17T01:20:38.214Z,None,2022-06-29T14:41:11.895Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,60.820,845436529,55,9.730959e+08,143271249.0,67.00,...,1.600000e+07,1.600000e+07,661.69,-90.79902,2021-05-18T21:19:59.514Z,26.020000,133.95307,2020-11-05T09:20:11.928Z,None,2022-06-29T14:40:58.713Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.078,207770559,154,2.694270e+08,53670652.0,1.15,...,2.429169e+08,2.500000e+08,23.38,-95.38790,2021-03-13T23:44:36.774Z,0.475381,126.79603,2020-11-04T14:53:53.560Z,None,2022-06-29T14:40:51.479Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,5686.810,179708728,170,2.087212e+08,51114765.0,6030.91,...,3.666600e+04,3.666600e+04,90787.00,-93.71029,2021-05-12T00:29:37.713Z,31.650000,17944.58693,2020-07-18T12:26:27.150Z,None,2022-06-29T14:41:41.840Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,23.350,31522511,518,5.591154e+07,3278032.0,25.08,...,2.074262e+06,2.393060e+06,2066.20,-98.87053,2021-03-20T19:33:19.740Z,17.340000,34.60576,2022-06-18T20:12:02.334Z,None,2022-06-29T14:41:40.822Z


### Marketcap ranking & top10 filtering

In [54]:
chosen_tokens = lt_passed.sort_values("market_cap", ascending=False).iloc[:10]

In [55]:
chosen_tokens

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.090,2323227432,27,5.089333e+09,132514384.0,5.33,...,1.000000e+09,1.000000e+09,44.92,-88.73185,2021-05-03T05:25:04.822Z,1.030000,391.31993,2020-09-17T01:20:38.214Z,None,2022-06-29T14:41:11.895Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,60.820,845436529,55,9.730959e+08,143271249.0,67.00,...,1.600000e+07,1.600000e+07,661.69,-90.79902,2021-05-18T21:19:59.514Z,26.020000,133.95307,2020-11-05T09:20:11.928Z,None,2022-06-29T14:40:58.713Z
SUSHI,sushi,sushi,Sushi,https://assets.coingecko.com/coins/images/1227...,1.078,207770559,154,2.694270e+08,53670652.0,1.15,...,2.429169e+08,2.500000e+08,23.38,-95.38790,2021-03-13T23:44:36.774Z,0.475381,126.79603,2020-11-04T14:53:53.560Z,None,2022-06-29T14:40:51.479Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,5686.810,179708728,170,2.087212e+08,51114765.0,6030.91,...,3.666600e+04,3.666600e+04,90787.00,-93.71029,2021-05-12T00:29:37.713Z,31.650000,17944.58693,2020-07-18T12:26:27.150Z,None,2022-06-29T14:41:41.840Z
ALCX,alchemix,alcx,Alchemix,https://assets.coingecko.com/coins/images/1411...,23.350,31522511,518,5.591154e+07,3278032.0,25.08,...,2.074262e+06,2.393060e+06,2066.20,-98.87053,2021-03-20T19:33:19.740Z,17.340000,34.60576,2022-06-18T20:12:02.334Z,None,2022-06-29T14:41:40.822Z


In [56]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [57]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

In [58]:
prices_data

,UNI,AAVE,SUSHI,YFI,ALCX
date,,,,,
2021-03-02,25.592699,386.507922,18.798040,34900.371111,1050.297397
2021-03-03,25.673868,392.581993,17.925825,33400.696195,991.795750
2021-03-04,25.486634,397.858343,17.517664,33881.025287,1376.135328
2021-03-05,28.368215,376.610205,17.408828,32560.148658,1139.907512
2021-03-06,28.308459,369.647205,16.690631,32137.453888,1167.196998
...,...,...,...,...,...
2022-06-25,5.508927,72.303936,1.292898,7123.154855,27.360469
2022-06-26,5.629472,69.962787,1.250895,7059.556445,27.590712
2022-06-27,5.607490,65.686644,1.148185,6470.666563,25.136293


In [59]:
marketcaps

,UNI,AAVE,SUSHI,YFI,ALCX
date,,,,,
2021-03-02,7.871645e+09,4.751222e+09,2.362758e+09,1.237769e+09,3.338203e+07
2021-03-03,7.938771e+09,4.878411e+09,2.273568e+09,1.188119e+09,3.126962e+07
2021-03-04,7.933740e+09,4.948727e+09,2.240011e+09,1.210977e+09,4.418676e+07
2021-03-05,8.803390e+09,4.684718e+09,2.211941e+09,1.156134e+09,3.647704e+07
2021-03-06,8.703822e+09,4.580840e+09,2.123790e+09,1.153659e+09,3.716741e+07
...,...,...,...,...,...
2022-06-25,2.526544e+09,9.951523e+08,2.504318e+08,2.259435e+08,3.707475e+07
2022-06-26,2.575548e+09,9.741046e+08,2.400655e+08,2.222005e+08,3.727336e+07
2022-06-27,2.569357e+09,9.142759e+08,2.202618e+08,2.042557e+08,3.396852e+07


## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [60]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [61]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [62]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [63]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [64]:
adjusted_weights = adjust_weights(weights, max_weight=0.3) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.005)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.3)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
adjusted_weights

,UNI,AAVE,SUSHI,YFI,ALCX
date,,,,,
2021-03-02,0.3,0.3,0.262490,0.137510,0.000000
2021-03-03,0.3,0.3,0.262712,0.137288,0.000000
2021-03-04,0.3,0.3,0.256355,0.138588,0.005057
2021-03-05,0.3,0.3,0.262695,0.137305,0.000000
2021-03-06,0.3,0.3,0.259200,0.140800,0.000000
...,...,...,...,...,...
2022-06-25,0.3,0.3,0.195097,0.176020,0.028883
2022-06-26,0.3,0.3,0.192230,0.177924,0.029846
2022-06-27,0.3,0.3,0.192164,0.178200,0.029635


In [65]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [66]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<76]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [67]:
last_weights = adjusted_weights.iloc[-1].dropna()
last_weights

UNI      0.300000
AAVE     0.300000
SUSHI    0.200012
YFI      0.169468
ALCX     0.030520
Name: 2022-06-29 00:00:00, dtype: float64

In [68]:
converted_last_weights = convert_weights(last_weights)
converted_last_weights

UNI      76
AAVE     76
SUSHI    52
YFI      44
ALCX      7
Name: 2022-06-29 00:00:00, dtype: int64

## Summary table

In [69]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb]['address'] for symb in pdi.index]

pdi = pdi[pdi['weight'] > 0]
pdi = pdi.sort_values("market_cap",ascending=False)
pdi

,name,market_cap,price,weight,weight_converted,address
symbol,,,,,,
UNI,Uniswap,2234493683,4.907910,0.300000,76,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
AAVE,Aave,833355296,60.169959,0.300000,76,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
SUSHI,Sushi,208941956,1.083175,0.200012,52,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2
YFI,yearn.finance,177034684,5615.224862,0.169468,44,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e
ALCX,Alchemix,31882882,23.655504,0.030520,7,0xdbdb4d16eda451d0503b854cf79d55697f90c8df


# 